## 3.We now review k-fold cross-validation.
 (a) Explain how k-fold cross-validation is implemented. 

The k-fold cross validation is implemented by taking the amount of observations, n, and randomly splitting them into k, non-overlapping groups of length of approximately n/k. These groups acts as a validation set, and the remainder acts as a training set. The test error is then estimated by averaging the k resulting MSE estimates.

(b) What are the advantages and disadvantages of k-fold cross validation relative to: 
i. The validation set approach?
ii. LOOCV?


A disadvantages of the validation set approach relative to k-fold cross-validation is the validation estimate of the test error rate can be highly variable (depends on which observations are included in the training/validation set). Another disadvantage is that only a subset of the observations are used to fit the model, so the validation set error may overestimate the test error rate for the model fit on the entire data set.



LOOCV has less bias. We repeatedly fit the statistical learning method using training data that contains n-1 obs., i.e. almost all the data set is used LOOCV produces a less variable MSE. The validation approach produces different MSE when applied repeatedly due to randomness in the splitting process, while performing LOOCV multiple times will always yield the same results, because we split based on 1 obs. each time LOOCV is computationally intensive (disadvantage). We fit the each model n times.

## 5

In [1]:
# Import standard Python data science libraries

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
import seaborn as sns
sns.set()

# Import classes from scikit-learn for logistic regression, LDA, QDA, and KNN classification
# Import convenience function for computing confusion matrices 
# Import OneHotEncoder and StandardScaler for data pre-processing
# Import Pipeline, ColumnTransformer to encapsulate pre-processing heterogenous data and fitting
# into a single estimator
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

# Load StatsModels API
# Note that if we wish to use R-style formulas, then we would use the StatsModels Formula API
import statsmodels.api as sm
import statsmodels.formula.api as smf

Default = pd.read_csv("C:/Users/nihar/OneDrive/Desktop/Predictive Modeling/Data Sets/Default.csv")
Default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [2]:
Default.columns

Index(['default', 'student', 'balance', 'income'], dtype='object')

## 5(a)

In [3]:
# Split data into training and validation sets
train_data, validation_data = train_test_split(Default, test_size=0.3, random_state=123)

# Set a random seed for reproducibility
np.random.seed(1)

model = LogisticRegression()
model.fit(train_data[['income', 'balance']], train_data['default'])

# Predict on the validation set
validation_pred = model.predict(validation_data[['income', 'balance']])
validation_pred


array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

## 5(b)


In [4]:
# Predict on the test set
test_data = Default.drop(validation_data.index)
test_pred = model.predict(test_data[['income', 'balance']])

# Calculate the misclassification error on the test set
test_misclassification_error = 1 - accuracy_score(test_data['default'], test_pred)
test_misclassification_error

0.03400000000000003

In [5]:
# i. Split the sample set into a training set and a validation set
train_data, validation_data = train_test_split(Default, test_size=0.3, random_state=123)

# ii. Fit a multiple logistic regression model using only the training observations
model = LogisticRegression()
model.fit(train_data[['income', 'balance']], train_data['default'])

# iii. Obtain a prediction of default status for each individual in the validation set
validation_data['default'] = (validation_data['default'] == 'Yes').astype(int)

validation_pred_proba = model.predict_proba(validation_data[['income', 'balance']])
validation_pred_default = (validation_pred_proba[:, 1] > 0.5).astype(int)

# iv. Compute the validation set error
validation_error = 1 - accuracy_score(validation_data['default'], validation_pred_default)
validation_error

0.03266666666666662

## 5(c)

In [6]:
# Repeat the process three times
for i in range(3):
    # Split the sample set into a training set and a validation set
    train_data, validation_data = train_test_split(Default, test_size=0.3, random_state=i)
    
    # Fit a multiple logistic regression model using only the training observations
    model = LogisticRegression()
    model.fit(train_data[['income', 'balance']], train_data['default'])
    
    # Obtain a prediction of default status for each individual in the validation set
    validation_pred1 = model.predict(validation_data[['income', 'balance']])
    
    # Compute the validation set error
    validation_error1 = 1 - accuracy_score(validation_data['default'], validation_pred1)
    print(f'Iteration {i+1}: {validation_error1}')

Iteration 1: 0.02733333333333332
Iteration 2: 0.024666666666666615
Iteration 3: 0.023666666666666614


The validation set errors for each iteration are relatively low, ranging from approximately 0.0236 to 0.0273. This suggests that the logistic regression model is performing well on the validation sets across different splits of the data. The consistency in the validation set errors across the iterations indicates that the model is stable and not highly sensitive to the specific training-validation set splits. Overall, the results suggest that the logistic regression model using income and balance as predictors is effective in predicting default status on this dataset.

## 5(d)

In [7]:
# Add a dummy variable for student
Default['student_dummy'] = pd.get_dummies(Default['student'], drop_first=True)

# Split the dataset into training and validation sets
train_data, validation_data = train_test_split(Default, test_size=0.3, random_state=123)

# Fit a logistic regression model
model = LogisticRegression()
model.fit(train_data[['income', 'balance', 'student_dummy']], train_data['default'])

# Predict on the validation set
validation_pred = model.predict(validation_data[['income', 'balance', 'student_dummy']])

# Calculate the misclassification error
misclassification_error = 1 - accuracy_score(validation_data['default'], validation_pred)
misclassification_error

0.02733333333333332

The misclassification error for the logistic regression model that includes a dummy variable for student is approximately 0.0273, which is very similar to the misclassification error of approximately 0.0273 obtained from the model that only used income and balance as predictors. This suggests that including a dummy variable for student did not lead to a significant reduction in the test error rate compared to the model without the student dummy variable.

## 6

In [8]:


from bootstrapped import bootstrap as bs
from bootstrapped import stats_functions as bs_stats
from statsmodels.discrete.discrete_model import Logit

# Set a random seed
np.random.seed(2)

#Compute estimated standard errors using sm.GLM()
model = sm.GLM.from_formula('default ~ income + balance', family=sm.families.Binomial(), data=Default)
result = model.fit()
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                        Generalized Linear Model Regression Results                        
===========================================================================================
Dep. Variable:     ['default[No]', 'default[Yes]']   No. Observations:                10000
Model:                                         GLM   Df Residuals:                     9997
Model Family:                             Binomial   Df Model:                            2
Link Function:                               Logit   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:                -789.48
Date:                             Fri, 08 Mar 2024   Deviance:                       1579.0
Time:                                     15:14:02   Pearson chi2:                 6.95e+03
No. Iterations:                                  9   Pseudo R-squ. (CS):             0.1256
Covariance Type:                         nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.5405      0.435     26.544      0.000      10.688      12.393
income     -2.081e-05   4.99e-06     -4.174      0.000   -3.06e-05    -1.1e-05
balance       -0.0056      0.000    -24.835      0.000      -0.006      -0.005
==============================================================================
"""

## 6(b)

In [9]:
def boot_fn(Default, idx):
    model = smf.glm(formula='default ~ income + balance', data=Default.iloc[idx], family=sm.families.Binomial()).fit()
    return model.params

## 6(c)

In [10]:
def bootstrap(Default, n=1000):
    n_obs = len(Default)
    coefs = []
    for _ in range(n):
        idx = np.random.choice(n_obs, n_obs, replace=True)
        coef = boot_fn(Default, idx)
        coefs.append(coef)
    coefs = pd.DataFrame(coefs)
    return coefs.std()

bootstrap_se = bootstrap(Default)
bootstrap_se

Intercept    0.432966
income       0.000005
balance      0.000228
dtype: float64

## 6(d)

I see that the estimated standard errors obtained using the sm.GLM() function for the coefficients of income and balance are very small, with values of approximately 0.000005 and 0.000228, These small standard errors suggest that the estimated coefficients are very precise and that there is very little uncertainty in their values.

The estimated standard errors obtained using the bootstrap method are 0.432966 for the intercept, 0.000005 for income, and 0.000228 for balance. These values are larger than those obtained using the sm.GLM() function, indicating that the bootstrap method introduces more uncertainty into the estimates.

The bootstrap method provides a more realistic estimate of the uncertainty in the coefficient estimates compared to the sm.GLM() function, which may underestimate the true uncertainty. Therefore, in this case, the bootstrap method is likely to be more reliable for assessing the precision of the coefficient estimates.

## 9

In [11]:
Boston = pd.read_csv("C:/Users/nihar/OneDrive/Desktop/Predictive Modeling/Data Sets/Boston.csv")
Boston.head()

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [12]:
medv_mean = np.mean(Boston.medv)
medv_mean

22.532806324110677

## 9(b)

In [13]:
medv_std = np.std(Boston.medv, ddof=1)  # Sample standard deviation
n = len(Boston.medv)
se_mean = medv_std / np.sqrt(n)
se_mean

0.4088611474975351

## 9(c)

In [14]:
n_iterations = 1000
bootstrap_means = [np.mean(np.random.choice(Boston.medv, size=n, replace=True)) for _ in range(n_iterations)]
bootstrap_se_mean = np.std(bootstrap_means)
bootstrap_se_mean

0.4132671309486871

The bootstrap estimate of the standard error of the mean (µˆ) is approximately 0.420. This is close to the estimate obtained using the formula, indicating that the bootstrap method provides a reliable estimate of the standard error.

## 9(d)

In [15]:
conf_interval = [medv_mean - 2 * se_mean, medv_mean + 2 * se_mean]
conf_interval

[21.715084029115605, 23.35052861910575]

## 9(e)

In [16]:
medv_median = np.median(Boston.medv)
medv_median

21.2

## 9(f)

In [17]:
bootstrap_medians = [np.median(np.random.choice(Boston.medv, size=n, replace=True)) for _ in range(n_iterations)]
bootstrap_se_median = np.std(bootstrap_medians)
bootstrap_se_median

0.3762626609165461

The bootstrap estimate of the standard error of the median is approximately 0.388. This provides a measure of the variability of the median estimate

## 9(g)

In [18]:
medv_10th_percentile = np.percentile(Boston.medv, 10)
medv_10th_percentile

12.75

## 9(h)

In [19]:
bootstrap_10th_percentiles = [np.percentile(np.random.choice(Boston.medv, size=n, replace=True), 10) for _ in range(n_iterations)]
bootstrap_se_10th_percentile = np.std(bootstrap_10th_percentiles)
bootstrap_se_10th_percentile

0.5113515400387486

The bootstrap estimate of the standard error of the tenth percentile is approximately 0.501. This provides a measure of the variability of the tenth percentile estimate.